In [ ]:
from scipy.signal import decimate
import h5py
import numpy as np
import os

In [ ]:
def print_stats(filename_path):
    with h5py.File(filename_path, 'r') as f:
        dataset_name = next(iter(f.keys()))
        print(f"Loading dataset: {dataset_name!r}")
        data = f[dataset_name][()] 
       

        means=data.mean(axis=1)
        stds=data.std(axis=1)
        print(f"data.shape: {data.shape}")
        print(f"means: min={means.min():.3e}, max={means.max():.3e}")
        print(f"stds: min={stds.min():.3e}, max={stds.max():.3e}")

In [ ]:
filename_path = "fpd/Intra/train/rest_105923_1.h5"
print_stats(filename_path)

In [ ]:
def load_data(filename):
    with h5py.File(filename, 'r') as f:
        data = f[next(iter(f.keys()))][()]
    return data

def normalize(data):
    mean = data.mean(axis=1, keepdims=True)
    std = data.std(axis=1, keepdims=True)
    return (data - mean) / std

def downsample(data, factor):
    return decimate(data, factor, axis=1)

def process_and_save(input_dir, output_dir, factor):
    os.makedirs(output_dir, exist_ok=True)
    for file in os.listdir(input_dir):
        if not file.endswith('.h5'):
            continue
        path = os.path.join(input_dir, file)
        data = load_data(path)
        data = normalize(data)
        data = downsample(data, factor)
        with h5py.File(os.path.join(output_dir, file), 'w') as f:
            f.create_dataset(file.rsplit('.', 1)[0], data=data)

process_and_save("fpd/Intra/train", "fpd/Processed/train", 8)
process_and_save("fpd/Intra/test", "fpd/Processed/test", 8)

In [ ]:
filename_path = r"fpd\Processed\train\rest_105923_1.h5"
print_stats(filename_path)